In [0]:
import math
import tensorflow as tf
from tensorflow import keras
from keras import backend as K

In [0]:
# use PSNR for metric
def PSNR(y_true, y_pred):
    max_pixel = 1.0
    return (10.0 * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true), axis=-1)))) / 2.303

In [0]:
(x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = x_train, x_test

In [0]:
inputs = tf.keras.Input(shape=(None,None,3))
noise = tf.keras.layers.GaussianNoise(0.1, input_shape=(None,None,3))(inputs)
x = tf.keras.layers.Conv2D(64, (3, 3), padding='same', input_shape=(None,None,3))(noise)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Conv2D(64, (3, 3), padding='same', input_shape=(None,None,64))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Conv2D(64, (3, 3), padding='same', input_shape=(None,None,64))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Conv2D(64, (3, 3), padding='same', input_shape=(None,None,64))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Conv2D(3, (3, 3), padding='same', input_shape=(None,None,64))(x)
out = tf.keras.layers.Add()([noise, x])
model = keras.models.Model(inputs=inputs, outputs=out)
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss='mse',
    metrics=[PSNR]
)

In [0]:
model.summary()

In [0]:
model.fit(x_train, y_train, batch_size=32, epochs=100)

In [0]:
# give noise to test data manually
def give_noise(data):
  normal_inputs = tf.keras.Input(shape=(None,None,3))
  gaussian_noise = tf.keras.layers.GaussianNoise(0.1, input_shape=(None,None,3))(normal_inputs, training=True)
  transform_model = keras.models.Model(inputs=normal_inputs, outputs=gaussian_noise)
  output = transform_model.predict(data)
  return output

In [0]:
noised_x_test = give_noise(x_test)
loss, metric = model.evaluate(noised_x_test, y_test, batch_size=32)
print(loss, metric)

In [0]:
from PIL import Image
import numpy as np

In [0]:
image = Image.open('noisy.png')
target = np.array([np.array(image, dtype=np.float32)/255.0])

In [0]:
output = model.predict(target)
output_image = Image.fromarray(np.uint8(output[0]*255.0))
output_image.save("model3.png")